# Exploring the DPD Offer Set Calculation

In [57]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  27 10:00:31 2019

@author: Stefan
"""

Hier stellen wir die DPD Offer Set Calculation nach Bront et Al dar und reproduzieren dessen Table 3. Zunächst hatte ich Schwierigkeiten, die ich aber lösen könnte durch genaue Code Analyse und Entdeckung von Feinheiten zwischen Python und R.


In [ ]:
#  PACKAGES
# Data
import numpy as np
import pandas as pd

# Calculation and Counting
import itertools
import math
import copy

# Memoization
import functools

# Gurobi
from gurobipy import *
import re

# Plot
# import matplotlib.pyplot as plt

# Some hacks
import sys
from contextlib import redirect_stdout

from dat_Koch import get_data
from dat_Koch import get_data_without_variations
from dat_Koch import get_variations
from dat_Koch import get_capacities_and_preferences_no_purchase
from dat_Koch import get_preference_no_purchase


# %% HELPER-FUNCTIONS
def memoize(func):
    cache = func.cache = {}

    @functools.wraps(func)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]

    return memoizer


def get_offer_sets_all(products):
    """
    Generates all possible offer sets, starting with offering nothing.

    :param products:
    :return:
    """
    n = len(products)
    offer_sets_all = np.array(list(map(list, itertools.product([0, 1], repeat=n))))
    return offer_sets_all


# %% FUNCTIONS
@memoize
def customer_choice_individual(offer_set_tuple, preference_weights, preference_no_purchase):
    """
    For one customer of one customer segment, determine its purchase probabilities given one offer set and given
    that one customer of this segment arrived.

    Tuple needed for memoization.

    :param offer_set_tuple: tuple with offered products indicated by 1=product offered, 0=product not offered
    :param preference_weights: preference weights of one customer
    :param preference_no_purchase: no purchase preference of one customer
    :return: array of purchase probabilities ending with no purchase
    """

    if offer_set_tuple is None:
        ret = np.zeros_like(preference_weights)
        return np.append(ret, 1 - np.sum(ret))

    offer_set = np.asarray(offer_set_tuple)
    ret = preference_weights * offer_set
    ret = np.array(ret / (np.sum(ret) + preference_no_purchase))
    ret = np.append(ret, 1 - np.sum(ret))
    return ret


@memoize
def customer_choice_vector(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities):
    """
    From perspective of retailer: With which probability can he expect to sell each product (respectively non-purchase)

    :param offer_set_tuple: tuple with offered products indicated by 1=product offered
    :param preference_weights: preference weights of all customers
    :param preference_no_purchase: preference for no purchase for all customers
    :param arrival_probabilities: vector with arrival probabilities of all customer segments
    :return: array with probabilities of purchase ending with no purchase

    NOTE: probabilities don't have to sum up to one? BEACHTE: Unterschied zu (1) in Bront et all
    """
    probs = np.zeros(len(offer_set_tuple) + 1)
    for l in np.arange(len(preference_weights)):
        probs += arrival_probabilities[l] * customer_choice_individual(offer_set_tuple, preference_weights[l, :],
                                                                       preference_no_purchase[l])
    return probs


@memoize
def value_expected(capacities, t, preference_no_purchase):
    """
    Recursive implementation of the value function, i.e. dynamic program (DP) as described on p. 241.

    :param capacities:
    :param t: time to go (last chance for revenue is t=0)
    :param preference_no_purchase:
    :return: value to be expected and optimal policy
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations()
    T = len(times)

    offer_sets_to_test = get_offer_sets_all(products)

    max_index = 0
    max_val = 0

    if all(capacities == 0):
        return 0, None
    if any(capacities < 0):
        return -math.inf, None
    if t == T:
        return 0, None

    for offer_set_index in range(len(offer_sets_to_test)):
        offer_set = offer_sets_to_test[offer_set_index, :]
        probs = customer_choice_vector(tuple(offer_set), preference_weights, preference_no_purchase,
                                       arrival_probabilities)

        val = float(value_expected(capacities, t + 1, preference_no_purchase)[0])  # ohne "float" würde ein numpy array
        #  zurückgegeben, und das später (max_val = val) direkt auf val verknüpft (call by reference)
        for j in products:  # nett, da Nichtkaufalternative danach (products+1) kommt und so also nicht betrachtet wird
            p = float(probs[j])
            if p > 0.0:
                value_delta_j = delta_value_j(j, capacities, t + 1, A, preference_no_purchase)
                val += p * (revenues[j] - value_delta_j)

        if val > max_val:
            max_index = offer_set_index
            max_val = val
    return max_val, tuple(offer_sets_to_test[max_index, :])


def delta_value_j(j, capacities, t, A, preference_no_purchase):
    """
    For one product j, what is the difference in the value function if we sell one product.
    TODO: stört mich etwas, Inidices von t, eng verbandelt mit value_expected()

    :param j:
    :param capacities:
    :param t:
    :param preference_no_purchase:
    :return:
    """
    return value_expected(capacities, t, preference_no_purchase)[0] - \
        value_expected(capacities - A[:, j], t, preference_no_purchase)[0]


# %%
# FUNCTIONS for Bront et al
# helpers
def purchase_rate_vector(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities):
    """
    P_j(S) for all j, P_0(S) at the end

    :param offer_set_tuple: S
    :param preference_weights
    :param preference_no_purchase
    :param arrival_probabilities
    :return: P_j(S) for all j, P_0(S) at the end

    TODO: p wird hier normiert, also gegeben ein Customer kommt, zu welchem Segment gehört
    s. p. 772 in Bront et al.
    vgl. customer_choice_vector()
    Lsg. wird hier mit \lambda lam wieder ausgebügelt in CDLP()
    """
    probs = np.zeros(len(offer_set_tuple) + 1)
    p = arrival_probabilities/(sum(arrival_probabilities))
    for l in np.arange(len(preference_weights)):
        probs += p[l] * customer_choice_individual(offer_set_tuple, preference_weights[l, :],
                                                   preference_no_purchase[l])
    return probs


def revenue(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities, revenues):
    """
    R(S)

    :param offer_set_tuple: S
    :return: R(S)
    """
    return sum(revenues * purchase_rate_vector(offer_set_tuple, preference_weights, preference_no_purchase,
                                               arrival_probabilities)[:-1])


def quantity_i(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities, i, A):
    """
    Q_i(S)

    :param offer_set_tuple: S
    :param i: resource i
    :return: Q_i(S)
    """
    return sum(A[i, :] * purchase_rate_vector(offer_set_tuple, preference_weights, preference_no_purchase,
                                              arrival_probabilities)[:-1])


# %%
# FUNCTIONS for Bront et al
# functions
def CDLP(capacities, preference_no_purchase, offer_sets: np.ndarray):
    """
    Implements (4) of Bront et al. Needs the offer-sets to look at (N) as input.

    :param offer_sets: N
    :return: dictionary of (offer set, time offered),
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations()

    offer_sets = pd.DataFrame(offer_sets)
    lam = sum(arrival_probabilities)
    T = len(times)

    S = {}
    R = {}
    Q = {}
    for index, offer_array in offer_sets.iterrows():
        S[index] = tuple(offer_array)
        R[index] = revenue(tuple(offer_array), preference_weights, preference_no_purchase, arrival_probabilities,
                           revenues)
        temp = {}
        for i in resources:
            temp[i] = quantity_i(tuple(offer_array), preference_weights, preference_no_purchase,
                                 arrival_probabilities, i, A)
        Q[index] = temp

    try:
        m = Model()

        # Variables
        mt = m.addVars(offer_sets.index.values, name="t", lb=0.0)  # last constraint

        # Objective Function
        m.setObjective(lam * quicksum(R[s] * mt[s] for s in offer_sets.index.values), GRB.MAXIMIZE)

        mc = {}
        # Constraints
        for i in resources:
            mc[i] = m.addConstr(lam * quicksum(Q[s][i] * mt[s] for s in offer_sets.index.values), GRB.LESS_EQUAL,
                                capacities[i],
                                name="constraintOnResource")
        msigma = m.addConstr(quicksum(mt[s] for s in offer_sets.index.values), GRB.LESS_EQUAL, T)

        m.optimize()

        ret = {}
        pat = r".*?\[(.*)\].*"
        for v in m.getVars():
            if v.X > 0:
                match = re.search(pat, v.VarName)
                erg_index = match.group(1)
                ret[int(erg_index)] = (tuple(offer_sets.loc[int(erg_index)]), v.X)
                print(offer_sets.loc[int(erg_index)], ": ", v.X)

        dualPi = np.zeros_like(resources, dtype=float)
        for i in resources:
            dualPi[i] = mc[i].pi
        dualSigma = msigma.pi

        valOpt = m.objVal

        return ret, valOpt, dualPi, dualSigma

    except GurobiError:
        print('Error reported')


def column_MIP(preference_no_purchase, pi, w=0):  # pass w to test example for greedy heuristic
    """
    Implements MIP formulation on p. 775 lhs

    :param pi:
    :param w:
    :return: optimal tuple of products to offer
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations()

    K = 1/min(preference_no_purchase.min(), np.min(preference_weights[np.nonzero(preference_weights)]))+1

    if isinstance(w, int) and w == 0:  # 'and' is lazy version of &
        w = np.zeros_like(revenues, dtype=float)
        for j in products:
            w[j] = revenues[j] - sum(A[:, j]*pi)

    try:
        m = Model()

        mx = {}
        my = {}
        mz = {}

        # Variables
        for j in products:
            my[j] = m.addVar(0, 1, vtype=GRB.BINARY, name="y["+str(j)+"]")
        for l in customer_segments:
            mx[l] = m.addVar(0.0, name="x["+str(l)+"]")
            temp = {}
            for j in products:
                temp[j] = m.addVar(0.0, name="z["+str(l)+","+str(j)+"]")
            mz[l] = temp

        # Objective
        m.setObjective(quicksum(arrival_probabilities[l] * w[j] * preference_weights[l, j] * mz[l][j]
                                for l in customer_segments for j in products), GRB.MAXIMIZE)

        # Constraints
        mc1 = m.addConstrs((mx[l]*preference_no_purchase[l] +
                            quicksum(preference_weights[l, j]*mz[l][j] for j in products) == 1
                            for l in customer_segments), name="mc1")
        mc2 = m.addConstrs((mx[l] - mz[l][j] <= K - K*my[j] for l in customer_segments for j in products),
                           name="mc2")
        mc3 = m.addConstrs((mz[l][j] <= mx[l] for l in customer_segments for j in products), name="mc3")
        mc4 = m.addConstrs((mz[l][j] <= K*my[j] for l in customer_segments for j in products), name="mc4")

        m.optimize()

        y = np.zeros_like(revenues)
        for j in products:
            y[j] = my[j].x

        return tuple(y), m.objVal

    except GurobiError:
        print('Error reported')


def column_greedy(preference_no_purchase, pi, w=0, dataName=""):  # pass w to test example for greedy heuristic
    """
    Implements Greedy Heuristic on p. 775 rhs

    :param pi:
    :param w:
    :return: heuristically optimal tuple of products to offer
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations(dataName)

    # Step 1
    y = np.zeros_like(revenues)

    if isinstance(w, int) and w == 0:  # and is lazy version of &
        w = np.zeros_like(revenues, dtype=float)  # calculate the opportunity costs
        for j in products:
            w[j] = revenues[j] - sum(A[:, j]*pi)

    # Step 2
    Sprime = set(np.where(w > 0)[0])

    # Step 3
    value_marginal = np.zeros_like(w, dtype=float)
    for j in Sprime:
        for l in customer_segments:
            value_marginal[j] += preference_weights[l, j]/(preference_weights[l, j] + preference_no_purchase[l])
        value_marginal[j] *= w[j]

    jstar = np.argmax(value_marginal)
    v_new = value_marginal[jstar]

    S = {jstar}
    Sprime = Sprime-S

    # Step 4
    while True:
        v_akt = copy.deepcopy(v_new)  # deepcopy to be on the safe side
        v_temp = np.zeros_like(revenues, dtype=float)  # uses more space then necessary, but simplifies indices below
        for j in Sprime:
            for l in customer_segments:
                z = 0
                n = 0
                for p in S.union({j}):
                    z += w[p]*preference_weights[l, p]
                    n += preference_weights[l, p]
                n += preference_no_purchase[l]
                v_temp[j] += arrival_probabilities[l]*z/n
        jstar = np.argmax(value_marginal)  # argmax always returns index of first maxima (if there is > 1)
        v_new = value_marginal[jstar]
        if v_new > v_akt:
            S = S.union({jstar})
            Sprime = Sprime - {jstar}
        else:
            break

    # Step 5
    y[list(S)] = 1
    return tuple(y), v_new


# CDLP by column generation
def CDLP_by_column_generation(capacities, preference_no_purchase):
    """
    Implements Bront et als approach for CDLP by column generation as pointed out on p. 775 just above "5. Decomp..."

    :return:
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations()

    dual_pi = np.zeros(len(A))

    col_offerset, col_val = column_greedy(preference_no_purchase, dual_pi)
    if all(col_offerset == np.zeros_like(col_offerset)):
        print("MIP solution used to solve CDLP by column generation")
        col_offerset, col_val = column_MIP(preference_no_purchase, dual_pi)

    offer_sets = pd.DataFrame([np.array(col_offerset)])

    val_akt_CDLP = 0
    ret, val_new_CDLP, dual_pi, dual_sigma = CDLP(capacities, preference_no_purchase, offer_sets)

    while val_new_CDLP > val_akt_CDLP:
        val_akt_CDLP = copy.deepcopy(val_new_CDLP)  # deepcopy and new name to be on the safe side

        col_offerset, col_val = column_greedy(preference_no_purchase, dual_pi)
        if not offer_sets[(offer_sets == np.array(col_offerset)).all(axis=1)].index.empty:
            col_offerset, col_val = column_MIP(preference_no_purchase, dual_pi)
            if not offer_sets[(offer_sets == np.array(col_offerset)).all(axis=1)].index.empty:
                break  # nothing changed

        offer_sets = offer_sets.append([np.array(col_offerset)], ignore_index=True)
        ret, val_new_CDLP, dual_pi, dual_sigma = CDLP(capacities, preference_no_purchase, offer_sets)

    return ret, val_new_CDLP, dual_pi, dual_sigma

# %%
# DECOMPOSITION APPROXIMATION ALGORITHM

# leg level decomposition directly via (11)
@memoize
def value_leg_i_11(i, x_i, t, pi, preference_no_purchase):
    """
    Implements the table of value leg decomposition on p. 776

    :param i:
    :param x_i:
    :param t:
    :param pi:
    :return: optimal value, index of optimal offer set, tuple optimal offer set
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations()
    T = len(times)
    lam = sum(arrival_probabilities)

    if t == T+1:
        return 0, None, None, None
    elif x_i <= 0:
        return 0, None, None, None

    offer_sets_all = get_offer_sets_all(products)
    offer_sets_all = pd.DataFrame(offer_sets_all)

    val_akt = 0.0
    index_max = 0

    df2 = pd.DataFrame({"purchase_rates": [[0]] * offer_sets_all.__len__()})
    df3 = pd.DataFrame({"temps": [[0]] * offer_sets_all.__len__()})

    for index, offer_array in offer_sets_all.iterrows():
        temp = np.zeros_like(products, dtype=float)
        for j in products:
            if offer_array[j] > 0:
                temp[j] = (revenues[j] -
                           (value_leg_i_11(i, x_i, t+1, pi, preference_no_purchase)[0] -
                            value_leg_i_11(i, x_i-1, t+1, pi, preference_no_purchase)[0] - pi[i]) * A[i, j] -
                           sum(pi[A[:, j] == 1]))
        val_new = sum(purchase_rate_vector(tuple(offer_array), preference_weights,
                                           preference_no_purchase, arrival_probabilities)[:-1] * temp)
        if val_new > val_akt:
            index_max = copy.copy(index)
            val_akt = copy.deepcopy(val_new)

        df2.loc[index, "purchase_rates"] = [purchase_rate_vector(tuple(offer_array), preference_weights,
                                           preference_no_purchase, arrival_probabilities)]
        df3.loc[index, "temps"] = [temp]
    return lam * val_akt + value_leg_i_11(i, x_i, t+1, pi, preference_no_purchase)[0], \
        index_max, tuple(offer_sets_all.iloc[index_max]), df2.loc[index_max, "purchase_rates"], df3.loc[index_max, "temps"]


def displacement_costs_vector(capacities_remaining, preference_no_purchase, t, pi, beta=1):
    """
    Implements the displacement vector on p. 777

    :param capacities_remaining:
    :param t:
    :param pi:
    :param beta:
    :return:
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations()

    delta_v = 1.0*np.zeros_like(resources)
    for i in resources:
        delta_v[i] = beta * (value_leg_i_11(i, capacities_remaining[i], t, pi, preference_no_purchase)[0] -
                             value_leg_i_11(i, capacities_remaining[i] - 1, t, pi, preference_no_purchase)[0]) + \
                     (1-beta) * pi[i]
    return delta_v


def calculate_offer_set(capacities_remaining, preference_no_purchase, t, pi, beta=1, dataName=""):
    """
    Implements (14) on p. 777

    :param capacities_remaining:
    :param t:
    :param pi:
    :param beta:
    :return: index of optimal offer set, optimal offer set (the products)
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations(dataName)
    lam = sum(arrival_probabilities)

    val_akt = 0
    index_max = 0

    offer_sets_all = get_offer_sets_all(products)
    offer_sets_all = pd.DataFrame(offer_sets_all)

    displacement_costs = displacement_costs_vector(capacities_remaining, preference_no_purchase, t + 2, pi, beta)

    for index, offer_array in offer_sets_all.iterrows():
        val_new = 0
        purchase_rate = purchase_rate_vector(tuple(offer_array), preference_weights,
                                             preference_no_purchase, arrival_probabilities)
        for j in products:
            if offer_array[j] > 0 and all(capacities_remaining - A[:, j] >= 0):
                val_new += purchase_rate[j] * \
                           (revenues[j] - sum(displacement_costs*A[:, j]))
        val_new = lam*val_new

        if val_new > val_akt:
            index_max = copy.copy(index)
            val_akt = copy.deepcopy(val_new)

    return index_max, products[np.array(offer_sets_all.iloc[[index_max]] == 1)[0]] + 1, offer_sets_all


In [5]:
# toy example for explaining stuff, check implementation of CDLP
n = 8
products = np.arange(n)
revenues = np.array([1200, 800, 500, 500, 800, 500, 300, 300], dtype=np.float)

m = 3
resources = np.arange(m)
capacities = np.array([10, 5, 5])

# capacity demand matrix A (rows: resources, cols: products)
# a_ij = 1 if resource i is used by product j
A = np.array([[0, 1, 1, 0, 0, 1, 1, 0],
              [1, 0, 0, 0, 1, 0, 0, 0],
              [0, 1, 0, 1, 0, 1, 0, 1]])

T = 30
times = np.arange(T)

L = 5
customer_segments = np.arange(L)
arrival_probabilities = np.array([0.15, 0.15, 0.2, 0.25, 0.25])
preference_weights = np.array([[5, 0, 0, 0, 8, 0, 0, 0],
                              [10, 6, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 8, 5, 0, 0],
                              [0, 0, 4, 0, 0, 0, 8, 0],
                              [0, 0, 0, 6, 0, 0, 0, 8]])
preference_no_purchase = np.array([2, 5, 2, 2, 2])

Here we go. 
Wir gehen langsam und von hinten vor. Auffallend war, dass wir Tabelle 3 aus Bront et al nicht replizieren konnten.

In [9]:
def print_top(df, p=0.9):
    df2 = df.loc[df.loc[:, "val"] > p * max(df.loc[:, "val"]), :]
    return df2.sort_values(["val"], ascending=False)

capacities_remaining = np.array([1,0,1])
pi = np.array([0, 1134.55, 500])
t = 27
beta = 1

dataName = "example0"

resources, \
products, revenues, A, \
customer_segments, preference_weights, arrival_probabilities, \
times = get_data_without_variations(dataName)
lam = sum(arrival_probabilities)

val_akt = 0
index_max = 0

offer_sets_all = get_offer_sets_all(products)
offer_sets_all = pd.DataFrame(offer_sets_all)
df2 = pd.DataFrame({"purchase_rates":[[0]]*offer_sets_all.__len__()})
displacement_costs = displacement_costs_vector(capacities_remaining, preference_no_purchase, t + 1, pi, beta)

for index, offer_array in offer_sets_all.iterrows():
    # check if it makes sense to consider the offer_array
    # it doesn't make sense, if product cannot be sold due to capacity reasons
    alright = True
    for j in products:
        if offer_array[j] > 0 and any(capacities_remaining - A[:, j] < 0):
            alright = False

    if alright:
        val_new = 0.0
        purchase_rate = purchase_rate_vector(tuple(offer_array), preference_weights,
                                             preference_no_purchase, arrival_probabilities)
        for j in products:
            if offer_array[j] > 0:
                val_new += purchase_rate[j] * \
                           (revenues[j] - sum(displacement_costs * A[:, j]))
        val_new = lam * val_new

        if val_new > val_akt:
            index_max = copy.copy(index)
            val_akt = copy.deepcopy(val_new)

        offer_sets_all.loc[index, "val"] = val_new
        df2.loc[index, "purchase_rates"] = [np.around(purchase_rate, 2)]
    else:
        offer_sets_all.loc[index, "val"] = 0.0


# offer_sets_all["purchase_rates"] = df2["purchase_rates"]

q = print_top(offer_sets_all)
q


,0,1,2,3,4,5,6,7,val
116,0,1,1,1,0,1,0,0,281.285392
118,0,1,1,1,0,1,1,0,274.586702
117,0,1,1,1,0,1,0,1,270.639377
119,0,1,1,1,0,1,1,1,263.940686
86,0,1,0,1,0,1,1,0,256.596309


Produkt 6 wird also zusätzlich mit angeboten, was im optimalen Offerset aus Bront et al nicht der Fall ist (Index hierfür: 112).

Ursachenanalyse führt uns zunächst zum Vergleich der Kaufwkeiten.

In [13]:
offer_sets_all["purchase_rates"] = df2["purchase_rates"]
offer_sets_all.loc[[116, 112], :]

     0  1  2  3  4  5  6  7         val  \
116  0  1  1  1  0  1  0  0  281.285392   
112  0  1  1  1  0  0  0  0  221.477535   

                                        purchase_rates  
116  [[0.0, 0.08, 0.17, 0.19, 0.0, 0.14, 0.0, 0.0, ...  
112  [[0.0, 0.08, 0.17, 0.19, 0.0, 0.0, 0.0, 0.0, 0...  


116    [[0.0, 0.08, 0.17, 0.19, 0.0, 0.14, 0.0, 0.0, ...
112    [[0.0, 0.08, 0.17, 0.19, 0.0, 0.0, 0.0, 0.0, 0...
Name: purchase_rates, dtype: object

In [15]:
a = offer_sets_all.loc[116, "purchase_rates"]
a

[array([0.  , 0.08, 0.17, 0.19, 0.  , 0.14, 0.  , 0.  , 0.42])]

In [16]:
b = offer_sets_all.loc[112, "purchase_rates"]
b

[array([0.  , 0.08, 0.17, 0.19, 0.  , 0.  , 0.  , 0.  , 0.56])]

Die gesamte Kaufwkeit kommt also aus der Nichtkaufwkeit (Produkte 2,3,4 werden nicht kannabalisiert). Dies ergibt Sinn, da gegeben der Segmentdefinitionen nur Segment 3 an Produkt 6 interessiert ist, dieses sich aber für keines der Produkte 2,3,4 interessiert.

Die Ursache für unser Problem muss also im Displacement-Cost Vektor liegen. Los geht die Spurensuche. Wir betrachten das gegebene Szenario.

In [21]:
print(t)
print(pi)
print(beta)
print(capacities_remaining)

27
[   0.   1134.55  500.  ]
1
[1 0 1]


In [18]:
displacement_costs = displacement_costs_vector(capacities_remaining, preference_no_purchase, t + 1, pi, beta)
displacement_costs

array([40.6725,  0.    , 40.6725])

Auffallend sind die geringen Kosten, die auch symmetrisch sind. Wir beginnen das gesamte von hinten her zu reproduzieren.

In [23]:
t = 31

for i in resources:
    print("In t = ", t, " hat Ressource ", i+1, " einen value von ", value_leg_i_11(i, capacities_remaining[i], t, pi, preference_no_purchase)[0])

In t =  31  hat Ressource  1  einen value von  0
In t =  31  hat Ressource  2  einen value von  0
In t =  31  hat Ressource  3  einen value von  0


In [42]:
t = 30

for i in resources:
    temp = value_leg_i_11(i, capacities_remaining[i], t, pi, preference_no_purchase)
    print("In t = ", t, " hat Ressource ", i+1, " einen value von ", temp[0])
    print("\t \t optimales Offerset: \t ", temp[2])
    print("\t \t Kaufwkeiten: \t", temp[3])
    print("\n")

In t =  30  hat Ressource  1  einen value von  13.557500000000008
	 	 optimales Offerset: 	  (1, 0, 0, 0, 0, 0, 0, 0)
	 	 Kaufwkeiten: 	 [array([0.20714286, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.79285714])]


In t =  30  hat Ressource  2  einen value von  0
	 	 optimales Offerset: 	  None
	 	 Kaufwkeiten: 	 None


In t =  30  hat Ressource  3  einen value von  13.557500000000008
	 	 optimales Offerset: 	  (1, 0, 0, 0, 0, 0, 0, 0)
	 	 Kaufwkeiten: 	 [array([0.20714286, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.79285714])]




One error found: Indexing in Python is different to R. I wanted to make logical indexing (via true / false values), where any value different from 0 is interpreted as true. Here is a reproduction of the discovered mistake:

In [53]:
print(pi)
print(A[:, 1])
print("Mistake: \t", pi[A[:, 1]])
print("Intended: \t", pi[A[:, 1] == 1])

[   0.   1134.55  500.  ]
[1 0 1]
Mistake: 	 [1134.55    0.   1134.55]
Intended: 	 [  0. 500.]


Adjustment done in code for value_leg_i_11().

In [55]:
t = 30
capacities_remaining = [0, 0, 1]

for i in resources:
    temp = value_leg_i_11(i, capacities_remaining[i], t, pi, preference_no_purchase)
    print("In t = ", t, " hat Ressource ", i+1, " einen value von ", temp[0])
    print("\t \t optimales Offerset: \t ", temp[2])
    print("\t \t Kaufwkeiten: \t", temp[3])
    print("\n")

In t =  30  hat Ressource  1  einen value von  0
	 	 optimales Offerset: 	  None
	 	 Kaufwkeiten: 	 None


In t =  30  hat Ressource  2  einen value von  0
	 	 optimales Offerset: 	  None
	 	 Kaufwkeiten: 	 None


In t =  30  hat Ressource  3  einen value von  313.9664502164502
	 	 optimales Offerset: 	  (0, 1, 1, 1, 0, 1, 0, 0)
	 	 Kaufwkeiten: 	 [array([0.        , 0.08181818, 0.16666667, 0.1875    , 0.        ,
       0.14285714, 0.        , 0.        , 0.42115801])]




Adjusting the implementation of calculate_offer_set() to calculate the displacement_costs with t+2, we can reproduce table 3 exactly.

In [58]:
remaining_capacity = np.array([[2,2,1],
                               [2,1,2],
                               [1,2,2],
                               [2,1,1],
                               [1,2,1],
                               [1,1,2],
                               [2,1,0],
                               [2,0,1],
                               [1,2,0],
                               [0,2,1],
                               [1,0,2],
                               [0,1,2],
                               [1,1,1],
                               [1,1,0],
                               [1,0,1],
                               [0,1,1],
                               [1,0,0],
                               [0,1,0],
                               [0,0,1]])
preference_no_purchase = get_preference_no_purchase()
df = pd.DataFrame(index=np.arange(len(remaining_capacity)), columns=['rem cap', 'offer set'])
for indexi in np.arange(len(df)):
    df.loc[indexi] = [remaining_capacity[indexi], calculate_offer_set(remaining_capacity[indexi],
                                                                      preference_no_purchase, 27,
                                                                      np.array([0, 1134.55, 500]))[1]]
print(df)

      rem cap        offer set
0   [2, 2, 1]     [1, 3, 4, 5]
1   [2, 1, 2]  [1, 2, 3, 4, 6]
2   [1, 2, 2]     [1, 3, 4, 5]
3   [2, 1, 1]        [1, 3, 4]
4   [1, 2, 1]     [1, 3, 4, 5]
5   [1, 1, 2]  [1, 2, 3, 4, 6]
6   [2, 1, 0]           [1, 3]
7   [2, 0, 1]        [2, 3, 4]
8   [1, 2, 0]        [1, 3, 5]
9   [0, 2, 1]        [1, 4, 5]
10  [1, 0, 2]     [2, 3, 4, 6]
11  [0, 1, 2]           [1, 4]
12  [1, 1, 1]        [1, 3, 4]
13  [1, 1, 0]           [1, 3]
14  [1, 0, 1]        [2, 3, 4]
15  [0, 1, 1]           [1, 4]
16  [1, 0, 0]              [3]
17  [0, 1, 0]              [1]
18  [0, 0, 1]              [4]


In [59]:
remaining_capacity2 = np.array([[3, 2, 2],
                               [2, 3, 2],
                               [2, 2, 3],
                               [3, 2, 1],
                               [3, 1, 2],
                               [2, 3, 1],
                               [1, 3, 2],
                               [2, 1, 3],
                               [1, 2, 3],
                               [3, 1, 1],
                               [1, 3, 1],
                               [1, 1, 3],
                               [3, 1, 0],
                               [3, 0, 1],
                               [1, 3, 0],
                               [0, 3, 1],
                               [1, 0, 3],
                               [0, 1, 3],
                               [2, 2, 2]])
preference_no_purchase = get_preference_no_purchase()
df2 = pd.DataFrame(index=np.arange(len(remaining_capacity2)), columns=['rem cap', 'offer set'])
for indexi in np.arange(len(df2)):
    df2.loc[indexi] = [remaining_capacity2[indexi], calculate_offer_set(remaining_capacity2[indexi],
                                                                      preference_no_purchase, 27,
                                                                      np.array([0, 1134.55, 500]))[1]]
print(df2)

      rem cap        offer set
0   [3, 2, 2]     [1, 3, 4, 5]
1   [2, 3, 2]     [1, 3, 4, 5]
2   [2, 2, 3]  [1, 2, 3, 4, 5]
3   [3, 2, 1]     [1, 3, 4, 5]
4   [3, 1, 2]  [1, 2, 3, 4, 6]
5   [2, 3, 1]     [1, 3, 4, 5]
6   [1, 3, 2]     [1, 3, 4, 5]
7   [2, 1, 3]  [1, 2, 3, 4, 6]
8   [1, 2, 3]     [1, 3, 4, 5]
9   [3, 1, 1]     [1, 3, 4, 6]
10  [1, 3, 1]     [1, 3, 4, 5]
11  [1, 1, 3]  [1, 2, 3, 4, 6]
12  [3, 1, 0]           [1, 3]
13  [3, 0, 1]     [2, 3, 4, 6]
14  [1, 3, 0]        [1, 3, 5]
15  [0, 3, 1]        [1, 4, 5]
16  [1, 0, 3]     [2, 3, 4, 6]
17  [0, 1, 3]           [1, 4]
18  [2, 2, 2]     [1, 3, 4, 5]
